Imports

In [1]:
import torch
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import cv2
import PIL
from PIL import Image
import copy
import random as rand
import math

import Dataloader

Draw bounding box in image

In [2]:
def showImageBB(image):
    if isinstance(image[0], PIL.JpegImagePlugin.JpegImageFile):
        img = transform(image[0])
    else:
        img = image[0]    
    try:
        # img = img / 2 + 0.5  # unnormalize
        npimg = img.numpy()
        npimg = np.transpose(npimg, (1, 2, 0))
    except:
        npimg = img
    fig, ax = plt.subplots()
    ax.imshow(npimg)
    for box in image[1]['boxes']:
        rect = patches.Rectangle((box[0], box[1]), box[2], box[3], linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    plt.show()

Read in Images

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

dataset = Dataloader.PassionfruitDataset(
    '/media/marcus/b18c7b58-f10e-4d8d-a377-a2cf5a589ad5/marcus/UNI/Honours/Semester 2/COS 711/Assignments/Ass 3/Data/Train_Images/Train_Images_(augmented)',
    '/media/marcus/b18c7b58-f10e-4d8d-a377-a2cf5a589ad5/marcus/UNI/Honours/Semester 2/COS 711/Assignments/Ass 3/Data/Train.csv',
    None)


Gaussian Noise

In [4]:
def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 1
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss

        return noisy

    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 0.8 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        
        return noisy

    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        
        return noisy

In [5]:
def addNoise(pattern, opt):
    change = True
    noise_type = {0: 'gauss', 1: 'poisson', 2: 'speckle', 4: None}
    noise = noise_type[opt]
    
    newPattern = copy.deepcopy(pattern)
    img = np.array(newPattern[0])

    if noise != None:
        img = noisy(noise, img)
        img = (img).astype('int')

    else:
        change = False
    
    return img, newPattern, change    
    

Rotation

In [6]:
def rotate(origin, point, angle):
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy

In [7]:
def rotateImage(pattern, opt):
    change = True
    # Create a copy of the pattern
    newPattern = copy.deepcopy(pattern)
    
    # Get create rotation options
    rot_opt = {0: cv2.ROTATE_90_CLOCKWISE, 1: cv2.ROTATE_180, 2: cv2.ROTATE_90_COUNTERCLOCKWISE, 3: None}
    rot_opt_degree = {0: 90, 1: 180, 2: 270}
    rot = rot_opt[opt]


    img = np.array(newPattern[0])

    if rot != None:
        # Rotate Image
        img = cv2.rotate(img, rot)

        boxes = []
        row, col, chn = img.shape
        centre = [row/2, col/2]
        for box in newPattern[1]['boxes']:
            box = np.array(box)
            if rot == 0:
                xmin, ymin = rotate([centre[0], centre[1]],[box[0], box[1]], np.deg2rad(rot_opt_degree[rot]))
                width, height = [box[3], box[2]]
                xmin = xmin - width

            elif rot == 1:
                xmin, ymin = rotate([centre[0], centre[1]],[box[0], box[1]], np.deg2rad(rot_opt_degree[rot]))
                width, height = [box[2], box[3]]
                xmin, ymin = [xmin-width, ymin-height]

            else:
                xmin, ymin = rotate([centre[0], centre[1]],[box[0], box[1]], np.deg2rad(rot_opt_degree[rot]))
                width, height = [box[3], box[2]]
                ymin = ymin-height
            
            boxes.append([xmin,ymin,width,height])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        newPattern[1]['boxes'] = boxes

    else:
        change = False
    
    return img, newPattern, change


Flip

In [8]:
def flipImage(pattern, opt):
    change = True
    # Create a copy of the pattern
    newPattern = copy.deepcopy(pattern)
    
    # Get create rotation options
    flip  = opt

    img = np.array(newPattern[0])

    if flip != 2:
        # Rotate Image
        img = cv2.flip(img, flip)

        boxes = []
        row, col, chn = img.shape
        centre = [row/2, col/2]
        for box in newPattern[1]['boxes']:
            box = np.array(box)

            if flip == 0:
                xmin, ymin = [box[0], col-box[1]-box[3]]
                width, height = [box[2], box[3]]

            elif flip == 1:
                xmin, ymin = [row-box[0]-box[2], box[1]]
                width, height = [box[2], box[3]]

            else:
                xmin, ymin = [row-box[0]-box[2], col-box[1]-box[3]]
                width, height = [box[2], box[3]]
            
            boxes.append([xmin,ymin,width,height])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        newPattern[1]['boxes'] = boxes

    else:
        change = False
    
    return img, newPattern, change

Other data augmentation methods

In [9]:
count = 0


for pattern in dataset:
    # Add noise
    img, newPattern, change = addNoise(pattern, 0)
    dataset.addToDataset(img, newPattern, '_N0')

    img, newPattern, change = addNoise(pattern, 1)
    dataset.addToDataset(img, newPattern, '_N1')

    img, newPattern, change = addNoise(pattern, 2)
    dataset.addToDataset(img, newPattern, '_N2')

    # Rotate
    img, newPattern, change = rotateImage(pattern, 0)
    dataset.addToDataset(img, newPattern, '_R0')
    
    img, newPattern, change = rotateImage(pattern, 1)
    dataset.addToDataset(img, newPattern, '_R1')
    
    img, newPattern, change = rotateImage(pattern, 2)
    dataset.addToDataset(img, newPattern, '_R2')

    # Flip
    img, newPattern, change = flipImage(pattern, -1)
    dataset.addToDataset(img, newPattern, '_F0')

    img, newPattern, change = flipImage(pattern, 0)
    dataset.addToDataset(img, newPattern, '_F1')

    img, newPattern, change = flipImage(pattern, 1)
    dataset.addToDataset(img, newPattern, '_F2')